In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
ref = pd.read_excel("etwas.xlsx")
ref = ref.astype('str')
cols = ref.columns
for col in cols[4:]:
    if not ref[col].dtype == 'float64':
        ref[col] = ref[col].str.removeprefix('<')
ref.replace("NR", None, inplace = True)
ref
ref.iloc[:, 4:] = ref.iloc[:, 4:].astype('float32')
ref['Tissue'].replace('WM', 'MT', inplace = True)
ref['Tissue'].replace('L', 'Liver', inplace = True)
ref.drop(columns=['Ag'], inplace=True)
cols = ref.columns.to_list()
cols[5:] = ["Arsenic", "Cadmium", "Chromium", "Copper", "Mercury", "Zinc", "Sodium"]
ref.columns = cols
ref

In [ ]:
data = pd.read_excel("matlab_XLR.xlsx", sheet_name="Results Summary")
vn = data.columns
lt_track = np.zeros(data.shape, dtype = "bool")
for i in np.arange(3, len(vn)):
    lt_track[:, i] = ~(data[vn[i]].str.removeprefix("<") == data[vn[i]])
    temp = data[vn[i]].str.removeprefix("<")
    data[vn[i]] = temp
    
data.head()

In [ ]:
elements = ref.columns[5:]

x = data.columns[3:]
# seperating liver and mt using columns contatins the word "Liver"
Liver_id = x.str.contains("Liver")

id_13 = x.str.contains("#13")
colorl = x.to_numpy().copy()
colorl[Liver_id] = 'gray' # -> liver color
colorl[~Liver_id] = 'darksalmon' # -> muscle color
colorl[id_13] = 'seagreen' # -> 13 color
ticklabel = x.str.removeprefix('White Sturgeon (WS #')
ticklabel = ticklabel.str.replace(')','', regex=False)
ticklabel = ticklabel.str.replace('MT','',regex=False) 
ticklabel = ticklabel.str.replace('Liver','',regex=False) 

for tissue, t_name in zip(['MT', 'Liver'], ['Muscle', "Liver"]):
    if tissue == 'MT':
        temp = ~Liver_id
    else:
        temp = Liver_id
    for i in np.arange(len(elements)):
        plt.figure(figsize=(10, 8), facecolor= 'white')
        # obtaining related data
        values = np.array(data[np.logical_and(data["Analyte"] == elements[i], data["Units"] == "mg/kg wwt")].to_numpy()[:, 3:].ravel(), dtype=float)

        # finding nans to ignore in future
        nan_id = ref.loc[:, elements[i]].isna()
        
        # bar without 13
        id = temp & (~id_13)
        n = len(x[id])
        plt.bar(np.arange(n), values[id], color = colorl[id], label="Nechako mass mortality")
        
        # plotting just 13
        id = temp & (id_13)
        plt.bar(np.arange(n, n+1), values[id], color = colorl[id], label="juvenile by-catch")
        
        # plotting mean
        id = temp & (~id_13)
        plt.plot(np.arange(n), np.mean(values[id]).repeat(values[id].shape), color = 'darkred', linestyle = "dashed", linewidth = 3, label = 'Nechako mass mortality average')
        n += 2
        for c in ref['color'].unique():
            y = ref.loc[(ref["Tissue"] == tissue) & (~nan_id) & (ref['color'] == c), [elements[i], 'Group']]
            if y.size > 0:
                plt.bar(np.arange(n, n + len(y)), y.iloc[:, 0].to_numpy('float'), color = c, label = f"literature: {y.iloc[:, 1].to_numpy('str')[0]}")
                n = n + len(y)
        y = ref.loc[(ref["Tissue"] == tissue) & (~nan_id), [elements[i]]].to_numpy('float')
        if y.size > 0:
            plt.plot(np.arange(n-len(y), n), np.mean(y).repeat(y.shape[0]), color = 'teal', linestyle = "dashed", linewidth = 3, label = 'literature average')
        plt.legend(fontsize = 15, bbox_to_anchor=(1, 1), loc='upper left')
        plt.title(f'{elements[i]} in {t_name}', fontsize = 25, loc='left') # {t_name} is tissue name
        plt.xlabel("fish_ID", fontsize = 25)
        plt.ylabel("mg/kg wet weight", fontsize = 25)
        plt.xticks(ticks = np.arange(n),
                   labels = ticklabel[temp].to_list() + [''] + ref.loc[(ref["Tissue"] == tissue) & (~nan_id), 'Fish'].to_list(), rotation = 40, ha = "right", fontsize = 15)
        plt.yticks(fontsize = 15)
        if not os.path.exists("./pics/etwas/"):
            os.makedirs("./pics/etwas/")
        plt.savefig("./pics/etwas/"+elements[i]+'_'+tissue+".png")

## Only Muscle and Liver

In [ ]:
ref

In [ ]:
elements = ref.columns[5:]

x = data.columns[3:]
# seperating liver and mt using columns contatins the word "Liver"
Liver_id = x.str.contains("Liver")
id_13 = x.str.contains("#13")
colorl = x.to_numpy().copy()
colorl[Liver_id] = 'gray' # -> liver color
colorl[~Liver_id] = 'darksalmon' # -> muscle color
colorl[id_13] = 'seagreen' # -> 13 color
ticklabel = x.str.removeprefix('White Sturgeon (WS #')
ticklabel = ticklabel.str.replace(')','', regex=False)
ticklabel = ticklabel.str.replace('MT','',regex=False) 
ticklabel = ticklabel.str.replace('Liver','',regex=False) 


for i in np.arange(len(elements)):
    plt.figure(figsize=(12, 12), facecolor= 'white')
    # obtaining related data
    values = np.array(data[np.logical_and(data["Analyte"] == elements[i], data["Units"] == "mg/kg wwt")].to_numpy()[:, 3:].ravel(), dtype=float)

    # finding nans to ignore in future
    nan_id = ref.loc[:, elements[i]].isna()
    temp = ~Liver_id
    # bar without 13
    id = temp & (~id_13)
    n = len(x[id])
    plt.bar(np.arange(n), values[id], color = colorl[id], label="Muscle")
    
    # plotting just 13
    id = temp & (id_13)
    plt.bar(np.arange(n, n+1), values[id], color = colorl[id])
    
    # plotting mean
    id = temp & (~id_13)
    plt.plot(np.arange(n), np.mean(values[id]).repeat(values[id].shape), color = 'darkred', linestyle = "dashed", linewidth = 3, label = 'Muslce average')
    n += 2

    temp = Liver_id
    # bar without 13
    id = temp & (~id_13)
    plt.bar(np.arange(n, n+id.sum()), values[id], color = colorl[id], label="Liver")
    n = n + id.sum()

    # plotting just 13
    id = temp & (id_13)
    plt.bar(np.arange(n, n+1), values[id], color = colorl[id], label = "Juvenile by-catch")
    n = n+1

    # plotting mean
    id = temp & (~id_13)
    plt.plot(np.arange(n-id.sum(), n)-1, np.mean(values[id]).repeat(values[id].shape), color = 'dimgrey', linestyle = "dashed", linewidth = 3, label = 'Liver average')
    

    plt.legend(fontsize = 15)
    plt.title(f'{elements[i]}', fontsize = 30, loc = 'left')
    plt.xlabel("fish_ID", fontsize = 25)
    plt.ylabel("mg/kg wet weight", fontsize = 20)
    plt.xticks(ticks = np.arange(n),
                labels = ticklabel[~temp].to_list() + [''] + ticklabel[temp].to_list(), fontsize = 20, fontweight = 'bold')
    plt.yticks(fontsize = 25)
    if not os.path.exists("./pics/etwas/barPlot+13/"):
        os.makedirs("./pics/etwas/barPlot+13/")
    plt.savefig("./pics/etwas/barPlot+13/"+elements[i]+'_'+tissue+".png")